In [2]:
import pandas as pd
import datetime
import math
import workdays


class Project:
    def __init__(self, name, num_dwgs, num_units, activation_date=None, team=[]):
        self.name = name
        self.num_dwgs = num_dwgs
        self.analysis_days = max([1, math.ceil(num_dwgs*.03)])
        self.num_units = num_units
        self.team = team
        self.activation_date = activation_date
        self.start = None
        self.finish = None
        self.submittals = []
        self.num_teammates = None

        self.holidays =  [datetime.date(2019, 12, 25),
                          datetime.date(2019,1,1 ),
                          datetime.date(2019, 5,27),
                          datetime.date(2019, 6,4),
                          datetime.date(2019, 9, 2),
                          datetime.date(2019, 11, 28)]
        self.workdays = None
        self.dwg_per_day = 2
        self.dwg_max_per_sub = 60
        self.QC_days = max([3, math.ceil(num_dwgs*.05)])
        self.drawing_difficulty_factor = 1
        self.expected_approval_date = None
        self.expected_book_release = None
        self.calc_workdays()
        self.calc_submittals()
        self.calc_last_sub()
        
    def calc_num_teammates(self):
        self.num_teammates = len(self.team)
        return self.num_teammates
        
    def calc_workdays(self):
        """Calculates the number of days the team assigned will need
            to finish drafting the project.  This function rounds up to the next
            whole day.  Uses the number of drawings per day defined above.  accounts
            for QC days and analysis days"""
        self.calc_num_teammates()
        self.workdays =  math.ceil(((self.num_dwgs*self.drawing_difficulty_factor)
                                    /(self.num_teammates*self.dwg_per_day)
                                    +self.QC_days+ self.analysis_days))
        return self.workdays
    
    def calc_submittals(self, start = None):
        num_sub = math.ceil(self.num_dwgs/self.dwg_max_per_sub)
        dwg_per = self.num_dwgs/num_sub
        duration_sub = math.ceil(self.workdays/num_sub)
        
        #default start is the activation day, proceeding dates will be determined 
        #by how long each submittal is
        if start is None:
            start = self.activation_date
        self.submittals = []
        for s in range(num_sub):

            # this calculates the last workday specified from a start date
            # plus some number of workdays accounting for (skipping) holidays
            # and weekends
            start = workdays.workday(start, days = duration_sub, holidays = self.holidays)
            
            self.submittals.append(start)
        
        last = self.submittals[-1]
        self.expected_approval_date = workdays.workday(last, days = 10, holidays = self.holidays)
        self.expected_book_release = workdays.workday(self.expected_approval_date, days = 2, holidays = self.holidays)
        
        
        
        return self.submittals
    
    def calc_last_sub(self):
        return self.submittals[-1]
       
    
    def report(self):
        def myformat(name, value):
            print( "{:<17} {:<20}".format(name, value))
        #print(f"Report Date: {datetime.datetime.now()}")
        myformat("Report Date:", str(datetime.datetime.now()))
        #print(f"Project: {self.name}\n{self.num_dwgs} Drawings\n{self.num_units} Units\n")
        myformat("Project:", self.name)
        myformat("Drawings:", self.num_dwgs)
        myformat("Units:", self.num_units)
        print()
        print(f"There will be {len(self.submittals)} submittals of plus or minus {math.ceil(self.num_dwgs/len(self.submittals))} Drawings")
        team_list = self.team
        team_list[-1] = "and " + team_list[-1]
        team = str(team_list).replace("[","").replace("]","").replace("'","")
        print(f"The work will be split between {team}")
        print()
        print("Timeline:")
        print(f"{self.activation_date.strftime('%b %d, %Y'):<17}{self.name} Activation")
        for i in range(len(self.submittals)):
            date = self.submittals[i]
            date = date.strftime('%b %d, %Y')
            print(f"{date:<17}Submittal Number {i+1}")
        cal_weeks = round((self.submittals[-1] - self.activation_date).days/7)
        
        print(f"{self.expected_approval_date.strftime('%b %d, %Y') :<17}Expected Approval Date ")
        print(f"{self.expected_book_release.strftime('%b %d, %Y'):<17}Expected Book Release Date ")
        print()
        print(f"There are {cal_weeks} calendar weeks between activation to the expected final submittal")
        
        dwg_per_day = math.ceil(self.num_dwgs/self.workdays)
        dwg_per_week = dwg_per_day * 5
        print(f"The team should target {dwg_per_week} drawings per work week and {dwg_per_day} drawings per work day to hit their goal")
        print(f"There are {self.workdays} work days for these drawings")
            
def main():        

    team_a = ["Andy", "Ashwini", "Cody", "Rick"]

    a = Project("PS 171", 157, 973, datetime.date(2019,5,6), team_a )
    a.report()
    
main()

Report Date:      2019-06-13 12:31:56.342995
Project:          PS 171              
Drawings:         157                 
Units:            973                 

There will be 3 submittals of plus or minus 53 Drawings
The work will be split between Andy, Ashwini, Cody, and Rick

Timeline:
May 06, 2019     PS 171 Activation
May 21, 2019     Submittal Number 1
Jun 07, 2019     Submittal Number 2
Jun 24, 2019     Submittal Number 3
Jul 08, 2019     Expected Approval Date 
Jul 10, 2019     Expected Book Release Date 

There are 7 calendar weeks between activation to the expected final submittal
The team should target 25 drawings per work week and 5 drawings per work day to hit their goal
There are 33 work days for these drawings


In [3]:





file = r"C:\Users\mkreidler\Desktop\Data\Drafting (4).xlsx"

def is_date(x):
    if type(x) == datetime.datetime:
        ##print("yes")
        return True
    else:
        #print(type(x))
        return False

df = pd.read_excel(file)
df = df[df["# Dwgs"] > 0]
df["activated"] = df["Activation Date"].apply(is_date)
df = df[df.activated == True]

for index, row in df.iterrows():
    team_a = ["Andy", "Ashwini", "Cody", "Rick"]
    project = Project(row["Task Name"], 
            row["# Dwgs"], 
            row["# units"], 
            row["Activation Date"].date(), 
            team_a)
            #row["Assigned To"] )
    project.report()
    print()
    print("******************************")
    print()
    

Report Date:      2019-06-13 12:31:56.445995
Project:          100 Hudson (need samples)
Drawings:         44.0                
Units:            533.0               

There will be 1 submittals of plus or minus 44 Drawings
The work will be split between Andy, Ashwini, Cody, and Rick

Timeline:
May 09, 2019     100 Hudson (need samples) Activation
May 24, 2019     Submittal Number 1
Jun 11, 2019     Expected Approval Date 
Jun 13, 2019     Expected Book Release Date 

There are 2 calendar weeks between activation to the expected final submittal
The team should target 20 drawings per work week and 4 drawings per work day to hit their goal
There are 11 work days for these drawings

******************************

Report Date:      2019-06-13 12:31:56.446995
Project:          114 Fifth Ave (Redesigning and Req. 2 courses of samples, accept CO winter work)
Drawings:         6.0                 
Units:            35.0                

There will be 1 submittals of plus or minus 6 Drawings
T

Units:            nan                 

There will be 1 submittals of plus or minus 3 Drawings
The work will be split between Andy, Ashwini, Cody, and Rick

Timeline:
Mar 18, 2019     Orpheum Theatre (CW fixing model/already submitted) Activation
Mar 25, 2019     Submittal Number 1
Apr 08, 2019     Expected Approval Date 
Apr 10, 2019     Expected Book Release Date 

There are 1 calendar weeks between activation to the expected final submittal
The team should target 5 drawings per work week and 1 drawings per work day to hit their goal
There are 5 work days for these drawings

******************************

Report Date:      2019-06-13 12:31:56.529995
Project:          PS 171 (M) (Need missing samples)
Drawings:         157.0               
Units:            973.0               

There will be 3 submittals of plus or minus 53 Drawings
The work will be split between Andy, Ashwini, Cody, and Rick

Timeline:
Mar 07, 2019     PS 171 (M) (Need missing samples) Activation
Mar 22, 2019     S

In [4]:
class Sequence():
    def __init__(self):
        self.projects = []
        self.team = None
        self.start = None
        self.finish = None
    
    def calc_sequence(self):
        start = None
        last_sub = None
        count = 0
        for p in self.projects:
            count += 1
            if count == 1:
                start = p.activation_date
                last_sub = p.calc_last_sub()
                p.report()
            else:
                start = last_sub
                p.calc_submittals(start = start)
                last_sub = p.calc_last_sub()
                p.report()
            print()
            print("*****************************")
            print()
        
team_a = ["Andy", "Ashwini", "Cody", "Rick"]

a1 = Project("PS 171", 157, 973, datetime.date(2019,5,6), team_a )
a2 = Project("100 Hudson", 157, 973, datetime.date(2019,5,12), team_a )
a3 = Project("123 Main st", 200, 50, datetime.date(2019,6,1), team_a )

s1 = Sequence()
s1.projects = [a1,a2,a3]
s1.calc_sequence()

team_b = ["Brian", "Carrie"]

b1 = Project("FCOCS", 100, 300, datetime.date(2019,5,6), team_b )
b2 = Project("Moynihan", 20, 973, datetime.date(2019,6,12), team_b )
b3 = Project("Sicily Towers", 12, 50, datetime.date(2019,4,13), team_b )

s2 = Sequence()
s2.projects = [b1,b2,b3]
s2.calc_sequence()




Report Date:      2019-06-13 12:31:56.545995
Project:          PS 171              
Drawings:         157                 
Units:            973                 

There will be 3 submittals of plus or minus 53 Drawings
The work will be split between Andy, Ashwini, Cody, and Rick

Timeline:
May 06, 2019     PS 171 Activation
May 21, 2019     Submittal Number 1
Jun 07, 2019     Submittal Number 2
Jun 24, 2019     Submittal Number 3
Jul 08, 2019     Expected Approval Date 
Jul 10, 2019     Expected Book Release Date 

There are 7 calendar weeks between activation to the expected final submittal
The team should target 25 drawings per work week and 5 drawings per work day to hit their goal
There are 33 work days for these drawings

*****************************

Report Date:      2019-06-13 12:31:56.545995
Project:          100 Hudson          
Drawings:         157                 
Units:            973                 

There will be 3 submittals of plus or minus 53 Drawings
The work will